## **1. Regroupement en sous-corpus**
Plutôt que de récupérer un tag qui aurait été assigné manuellement pour chaque lien, on va le créer en déconstruisant les URLs et en créant des sous-dossiers correspondant à différents sous-niveaux dans l'arborescence du site.  

À décider : jusqu'à quel niveau on regroupe (ça peut être un peu du cas par cas selon l'organisation du site)

Créer des sous-dossiers à l'intérieur

In [1]:
path = '../03-corpus/2-data/1-fr/'
acteur = "ciusss_centresud"

from pandas import *

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur  + '.csv', encoding='utf-8') as f:
    csv = read_csv(f, sep=',')[["Address", 'Title', "text"]]

csv

,Address,Title,text
0,https:////ciusss-centresudmtl.gouv.qc.ca//,CIUSSS du Centre-Sud-de-l&#039;Île-de-Montréal,Variole simienne Information essentielle conce...
1,https:////ciusss-centresudmtl.gouv.qc.ca//pers...,Personnel et retraités | CIUSSS du Centre-Sud-...,Équité salariale Pour en connaître davantage s...
2,https:////ciusss-centresudmtl.gouv.qc.ca//prop...,Services en anglais,À propos Services en anglais et autres langues...
3,https:////ciusss-centresudmtl.gouv.qc.ca//empl...,Emplois et stages | CIUSSS du Centre-Sud-de-l&...,Emplois et stages Emplois et stages Nous cherc...
4,https:////ciusss-centresudmtl.gouv.qc.ca//nous...,Nous joindre | CIUSSS du Centre-Sud-de-l&#039;...,Nous joindre Nous joindre Renseignements génér...
...,...,...,...
521,https:////ciusss-centresudmtl.gouv.qc.ca//actu...,Bilan annuel 2018,Actualités Bilan annuel des directeurs de la p...
522,https:////ciusss-centresudmtl.gouv.qc.ca//actu...,Bilan annuel 2017,Actualités Bilan annuel des directeurs de la p...
523,https:////ciusss-centresudmtl.gouv.qc.ca//actu...,Bilan annuel 2016,Actualités Bilan annuel des DPJ - Les mauvais ...
524,https:////ciusss-centresudmtl.gouv.qc.ca//medi...,Description du poste,Bénévoles recherchés pour des visites amicales...


In [2]:
liste = [lien.replace('//', "/") for lien in csv['Address'].tolist() if not 'actualites' in lien and not 'propos' in lien and not 'joindre' in lien and not 'communiques' in lien and not 'about' in lien and not 'nouvelles' in lien and not 'pdf' in lien and not 'gaspesie' in lien]
liste 

['https://ciusss-centresudmtl.gouv.qc.ca/',
 'https://ciusss-centresudmtl.gouv.qc.ca/personnel-et-retraites',
 'https://ciusss-centresudmtl.gouv.qc.ca/emplois-et-stages',
 'https://ciusss-centresudmtl.gouv.qc.ca/soins-et-services/demande-dacces-au-dossier-de-lusager-et-registre-de-vaccination',
 'https://ciusss-centresudmtl.gouv.qc.ca/partager-son-experience',
 'https://ciusss-centresudmtl.gouv.qc.ca/adresses-et-coordonnees',
 'https://ciusss-centresudmtl.gouv.qc.ca/mission-universitaire',
 'https://ciusss-centresudmtl.gouv.qc.ca/langue-des-signes-quebecoise-lsq-videos',
 'https://ciusss-centresudmtl.gouv.qc.ca/accessibilite',
 'https://ciusss-centresudmtl.gouv.qc.ca/informations-pratiques',
 'https://ciusss-centresudmtl.gouv.qc.ca/soins-et-services/consultation-avec-un-medecin-ou-une-infirmiere/acces-un-medecin-de-famille',
 'https://ciusss-centresudmtl.gouv.qc.ca/soins-et-services/carte-dassurance-maladie',
 'https://ciusss-centresudmtl.gouv.qc.ca/soins-et-services/consultation-avec-

In [3]:
"""
0 = http/s
1 = //
2 = domaine principal (ex. INSPQ)
3 = 1e sous-niveau
4 = 2e sous-niveau
5 = 3e sous-niveau
etc.
"""

liens = []
for i in range (2, 9): # On prend les liens de niveau 2 à 8
    try:
        liens += [lien for lien in liste if len(lien.split('/')) > i]
    except Exception as e:
        True

In [4]:
regroupements = {}

for lien in liens:
    for i in range(3,9):
        try:
            tag = lien.split('/')[i-1]
            if tag in regroupements:
                regroupements[tag].append(lien)
                
            else:
                if len(tag)>1:
                    regroupements[tag] = []

        except Exception as e:
            True

tags = list(regroupements.keys())

In [5]:
tags = [tag for tag in regroupements if len(regroupements[tag]) > 40]

In [6]:
tags

['ciusss-centresudmtl.gouv.qc.ca',
 'personnel-et-retraites',
 'emplois-et-stages',
 'soins-et-services',
 'informations-pratiques',
 'consultation-avec-un-medecin-ou-une-infirmiere',
 'consultation-avec-un-medecin-une-infirmiere-ou-un-travailleur-social',
 'mesures-speciales-pour-les-usagers-et-leurs-proches',
 'grossesse-et-soins-aux-bebes',
 'actualite',
 'personnel-infirmier-et-cardiorespiratoire',
 'professionnels',
 'etablissement',
 'consultation-avec-un-medecin-specialiste',
 'deficience-physique-sensorielle-langage-auditive-et-motrice',
 'grossesse',
 'aides-techniques-prothese-orthese-etc']

In [7]:
print('Ça nous donne {} tags.'.format(len(tags)))
tags

Ça nous donne 17 tags.


['ciusss-centresudmtl.gouv.qc.ca',
 'personnel-et-retraites',
 'emplois-et-stages',
 'soins-et-services',
 'informations-pratiques',
 'consultation-avec-un-medecin-ou-une-infirmiere',
 'consultation-avec-un-medecin-une-infirmiere-ou-un-travailleur-social',
 'mesures-speciales-pour-les-usagers-et-leurs-proches',
 'grossesse-et-soins-aux-bebes',
 'actualite',
 'personnel-infirmier-et-cardiorespiratoire',
 'professionnels',
 'etablissement',
 'consultation-avec-un-medecin-specialiste',
 'deficience-physique-sensorielle-langage-auditive-et-motrice',
 'grossesse',
 'aides-techniques-prothese-orthese-etc']

In [9]:
import os, shutil
from pathlib import Path
from slugify import slugify

# Créer un sous-dossier où on va stocker des nouveaux fichiers CSV correspondant aux sous-corpus
folder_path ='../03-corpus/2-sous-corpus/'

#Path(folder_path).mkdir(parents=True, exist_ok=True)


for tag in tags:
    file_name =  acteur + "_" + slugify(str(tag).strip('\n'))

    # Aller chercher les documents du sous-corpus dans le CSV du corpus principal et les copier dans un nouveau CSV
    try:
        data = csv[csv["Address"].str.match('.*[\-\_\/]'+tag+'.*')]
        data.to_csv(folder_path + file_name + ".csv") #encoding='ISO-8859-1'
    except Exception as e:
        print(tag, str(e))
                    

In [10]:
with open(folder_path + acteur + '_sous-corpus.txt', 'w') as f:
    for file in os.listdir(folder_path):
        f.write(file + '\n')

Après avoir manuellement indexé

In [ ]:
from pandas import read_csv
filepath = '../03-corpus/2-sous-corpus/0-semi-automatic_tags_' + acteur + '.csv'
with open(filepath, 'r', encoding = 'utf-8') as f:
    csv = read_csv(f, sep=';')
    
# csv = csv[csv['oui/non'] != 'x']
# csv = csv[csv['oui/non'] != 'X']
csv = csv[csv['Sous-corpus'] != 'x']
csv = csv[csv['Sous-corpus'] != 'X']

csv

,Regroupement dans l'arborescence du site,Sous-corpus
0,inesss_biologie-medicale-et-genomique.csv,Biologie médicale et génomique
1,inesss_cancerologie.csv,Cancérologie
2,inesss_covid-19.csv,COVID-19
6,inesss_medicaments.csv,Médicaments
7,inesss_medicaments-evaluation-aux-fins-dinscri...,Médicaments
11,inesss_produits-du-systeme-du-sang.csv,Produits du système du sang
13,inesss_projets-en-cours.csv,Projets
14,inesss_protocoles-medicaux-nationaux-et-ordonn...,Protocoles médicaux nationaux
15,inesss_publication.csv,Publications
16,inesss_publications.csv,Publications


In [ ]:
# tags = list(set([tag for tag in csv['oui/non'].tolist()]))
tags = list(set([tag for tag in csv['Sous-corpus'].tolist()]))

tags

['Médicaments',
 'Biologie médicale et génomique',
 'Services sociaux',
 'Projets',
 'COVID-19',
 'Cancérologie',
 'Publications',
 'Traumatologie',
 'Protocoles médicaux nationaux',
 'Produits du système du sang']

In [ ]:
regroupements = {tag : [] for tag in tags}
for tag in tags:
        tuples = csv[csv['Sous-corpus'] == tag]
        liste = [str(t).strip('\n') for t in tuples["Regroupement dans l'arborescence du site"].tolist()]
        regroupements[tag] = liste


In [ ]:
regroupements

{'Médicaments': ['inesss_medicaments.csv',
  'inesss_medicaments-evaluation-aux-fins-dinscription.csv',
  'inesss_rendez-vous-des-fabricants.csv'],
 'Biologie médicale et génomique': ['inesss_biologie-medicale-et-genomique.csv'],
 'Services sociaux': ['inesss_services-sociaux.csv'],
 'Projets': ['inesss_projets-en-cours.csv'],
 'COVID-19': ['inesss_covid-19.csv',
  'inesss_traitements-specifiques-a-la-covid-19.csv'],
 'Cancérologie': ['inesss_cancerologie.csv'],
 'Publications': ['inesss_publication.csv',
  'inesss_publications.csv',
  'inesss_repertoire-des-publications.csv'],
 'Traumatologie': ['inesss_traumatologie.csv'],
 'Protocoles médicaux nationaux': ['inesss_protocoles-medicaux-nationaux-et-ordonnances-associees.csv'],
 'Produits du système du sang': ['inesss_produits-du-systeme-du-sang.csv']}

In [ ]:
import pandas
from slugify import slugify

# Créer de nouveaux CSV contenant les données regroupées selon le tag qui leur est associé
path='../03-corpus/2-sous-corpus/'

for tag in regroupements:

    # Aller chercher les éléments correspondant au sous-corpus dans le csv principal et les copier dans un nouveau CSV 
    dfs = []
    path='../03-corpus/2-sous-corpus/'
    for doc in regroupements[tag]:
        with open(path + doc, 'r', encoding='utf-8') as f:
            csv = read_csv(f)
            dfs.append(csv)

    # Concatenate all data into one DataFrame
    try:
        path='../03-corpus/2-data/1-fr/' + acteur +'/'
        file_path = path + acteur + '_'  + slugify(str(tag)) + '.csv'
        big_frame = pandas.concat(dfs, ignore_index=True)
        big_frame.to_csv(file_path)
        print(tag + "\t" + str(len(big_frame)))

    except Exception as e:
        print(tag, str(e))



Médicaments	1099
Biologie médicale et génomique	11
Services sociaux	41
Projets	30
COVID-19	173
Cancérologie	25
Publications	566
Traumatologie	31
Protocoles médicaux nationaux	44
Produits du système du sang	15
